# **<font color="red">Events</font>**
Events are the fundamental units of information flow within the Agent Development Kit (ADK). They represent every significant occurrence during an agent's interaction lifecycle, from initial user input to the final response and all the steps in between.

## **<font color="blue">What Events Are and Why They Matter</font>**
An `Event` in ADK is an immutable record representing  a specific point in the agent's execution. It captures user messages, agent replies, requests to use tools (function calls), tool results, state changes, control signals, and errors.
Events are central to ADK's operation for several key reasons:
  - **Communication:** They serve as the standard message format between the user interface, the `Runner`, agents, the LLM, and tools. Everything flows as an `Event`.
  - **Signaling State & Artifact Changes:** Events carry instructions for state modifications and track artifact updates. Then `SessionService` uses these signals to ensure persistence. In Python changes are `signaled via event.actions.state_delta` and `event.actions.artifact_delta`.
  - **Control Flow:** Specific fields like `event.actions.transfer_to_agent` or `event.actions.escalate` act as signals that direct the framework, determining which agent runs next or if a loop should terminate.
  - **History & Observability:** The sequence of events recorded in `session.events` provides a complete, chronological history of an interaction, invaluable for debugging, auditing, and understanding agent behavior step-by-step
In the essence, the entire process, from a user's query to the agent's final answer, is orchestrated through the generation, interpretation, and processing of `Event` objects.

In [1]:
# ============================================================
# ADK EVENT DEMO (CONFIG + MODEL STRUCTURE APPLIED)
# ============================================================

import os
import asyncio
import uuid

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ------------------------------------------------------------
# CONFIG (Same structure as your Artifact demo)
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "event_demo_app"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOL (To Generate Tool Events)
# ============================================================

async def add_numbers(
    a: float,
    b: float,
    tool_context: ToolContext = None
):
    result = a + b
    return f"The result is {result}"


tools = [
    FunctionTool(func=add_numbers),
]


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="EventDemoAgent",
    model=MODEL,
    instruction="""
You are a helpful assistant.

If user asks for addition, ALWAYS call add_numbers tool.
Do not calculate manually.
""",
    tools=tools
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:

            print("\n--------------------------------------------------")
            print("Event ID       :", event.id)
            print("Author         :", event.author)
            print("Invocation ID  :", event.invocation_id)
            print("Timestamp      :", event.timestamp)
            print("Branch         :", event.branch)

            if event.content:
                print("Content        :", event.content)

            if event.actions:
                print("Actions        :", event.actions)

            if event.is_final_response():
                print("\n✅ Final Response:")
                print(event.content.parts[0].text)

    print("\n=== ADK EVENT FLOW DEMO ===\n")

    await invoke("What is 15 + 27?")


# ============================================================
# EXECUTION
# ============================================================

# For Jupyter:
await run_demo()

# For script mode:
# if __name__ == "__main__":
#     asyncio.run(run_demo())


=== ADK EVENT FLOW DEMO ===



D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:



--------------------------------------------------
Event ID       : 05816b84-f4ed-4328-8484-acf7bc7c865f
Author         : EventDemoAgent
Invocation ID  : e-9ae12a45-2321-463a-a1de-a59c2332bc94
Timestamp      : 1771570578.588251
Branch         : None
Content        : parts=[Part(
  function_call=FunctionCall(
    args={
      'a': 15,
      'b': 27
    },
    id='adk-81f8323f-6416-4004-be0b-bc81f2ea12b8',
    name='add_numbers'
  ),
  thought_signature=b"\n\xce\x01\x01\xbe>\xf6\xfb\xf1\xcc\xedx\xe3\xfe'k\xe07i\n\x9aM\nk\xf2\x13\xe8Z%\xb0\xf9\xc6k\xef\xf2:\xcc\xa1\xaa\xbb\xd8\xd2,\n&O\x0e\x98U\x8c\xebH\xc8O\xbc\xa3F\x91e\r\xf4\x96N\x08\xc7\xad?O\x18VU\xaal\xdc\x91<{\xd3\x10!\x00X~\xf0\xbb)8\x03O\xf0\xbc\x10\xa6b\xab[\x06...'
)] role='model'
Actions        : skip_summarization=None state_delta={} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={} requested_tool_confirmations={} compaction=None end_of_agent=None agent_state=None rewind_before_invocation_id=No

In [3]:
# ============================================================
# ADK EVENTS — FULL SIGNALING DEMO
# Demonstrates:
# 1. Communication via Events
# 2. state_delta
# 3. artifact_delta
# 4. Tool invocation events
# 5. Session event history
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "event_understanding_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOL 1 — STATE UPDATE TOOL
# Demonstrates event.actions.state_delta
# ============================================================

async def remember_value(
    key: str,
    value: str,
    tool_context: ToolContext = None
):
    # Signal state update
    tool_context.state[key] = value

    return f"Stored '{key}' = '{value}' in session state."


# ============================================================
# TOOL 2 — ARTIFACT SAVE TOOL
# Demonstrates artifact_delta
# ============================================================

async def save_note(
    filename: str,
    content: str,
    tool_context: ToolContext = None
):
    artifact_text = f"{datetime.utcnow()}\n\n{content}"

    part = types.Part.from_bytes(
        data=artifact_text.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return f"Artifact '{filename}' saved (version {version})"


# ============================================================
# REGISTER TOOLS
# ============================================================

tools = [
    FunctionTool(func=remember_value),
    FunctionTool(func=save_note),
]


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="EventAwareAgent",
    model=MODEL,
    instruction="""
You are a system-aware assistant.

If user says:
- Remember <key> as <value> → call remember_value
- Save note <filename> with <content> → call save_note
Otherwise respond normally.

Always use tools when appropriate.
""",
    tools=tools
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:

            print("\n=================================================")
            print("Event ID       :", event.id)
            print("Author         :", event.author)
            print("Invocation ID  :", event.invocation_id)
            print("Timestamp      :", event.timestamp)
            print("Branch         :", event.branch)

            # Communication layer
            if event.content:
                print("Content        :", event.content)

            # Signaling layer
            if event.actions:
                print("Actions        :", event.actions)

                if event.actions.state_delta:
                    print("🟢 STATE DELTA :", event.actions.state_delta)

                if event.actions.artifact_delta:
                    print("🟣 ARTIFACT DELTA :", event.actions.artifact_delta)

                if event.actions.transfer_to_agent:
                    print("🔄 TRANSFER TO :", event.actions.transfer_to_agent)

                if event.actions.escalate:
                    print("⚠ ESCALATION SIGNAL")

            if event.is_final_response():
                print("\n✅ Final Response:")
                print(event.content.parts[0].text)

    print("\n=========== EVENTS FULL DEMO ===========\n")

    # 1️⃣ Normal conversation (pure communication event)
    await invoke("Hello, how are you?")
    await asyncio.sleep(1)

    # 2️⃣ State update (state_delta event)
    await invoke("Remember project as ADK Research")
    await asyncio.sleep(1)

    # 3️⃣ Artifact creation (artifact_delta event)
    await invoke("Save note report.txt with This is the first report.")
    await asyncio.sleep(1)

    # ========================================================
    # 4️⃣ Observability — Print Full Session History
    # ========================================================

    print("\n\n=========== SESSION EVENT HISTORY ===========")

    session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    print(f"\nTotal Events Recorded: {len(session.events)}")

    for i, e in enumerate(session.events):
        print(f"{i+1}. {e.author} | {e.id}")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()

# For script mode:
# if __name__ == "__main__":
#     asyncio.run(run_demo())


=========== EVENTS FULL DEMO ===========


Event ID       : 9c56d9e3-4c47-4fd7-98db-a74dd9bd1389
Author         : EventAwareAgent
Invocation ID  : e-849941f2-7acb-4b63-b11c-bd8a5580b6ad
Timestamp      : 1771578527.890543
Branch         : None
Content        : parts=[Part(
  text="I'm doing well, thank you for asking!",
  thought_signature=b'\n\x8c\x03\x01\xbe>\xf6\xfb}\xed\x9dF\xa9\x19\x08\x94]\x80r\xb2\x18Q\x8f\x87\xa5W\xceB#2\xf2\x0exSP?\x00D\xc5\x1a\xe3>\x8a\xa70\xec\xe1\xc0\xc9:|\xb0\x0c\xca\xfcp\xbb{\xf8\xe2\xb6\xb0\xeeX\xb8\xcf\x85\x82\x15\x85\x816,^4W\x95\xb5+\xa9\xc1\x97.\xa8\xafX*\x89\xcd\x81\xd7N!\xd7\x08\x11R...'
)] role='model'
Actions        : skip_summarization=None state_delta={} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={} requested_tool_confirmations={} compaction=None end_of_agent=None agent_state=None rewind_before_invocation_id=None

✅ Final Response:
I'm doing well, thank you for asking!

Event ID       : 32435462-28af-4883-933c

In [13]:
import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "adk_event_master_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOL 1 — STATE TOOL (state_delta)
# ============================================================

async def remember(key: str, value: str, tool_context: ToolContext = None):
    tool_context.state[key] = value
    return f"Stored {key} = {value}"


# ============================================================
# TOOL 2 — ARTIFACT TOOL (artifact_delta)
# ============================================================

async def save_file(filename: str, content: str, tool_context: ToolContext = None):
    text = f"{datetime.utcnow()}\n\n{content}"

    part = types.Part.from_bytes(
        data=text.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return f"Saved {filename} version {version}"


memory_tool = FunctionTool(func=remember)
file_tool = FunctionTool(func=save_file)


# ============================================================
# MEMORY AGENT
# ============================================================

memory_agent = LlmAgent(
    name="MemoryAgent",
    model=MODEL,
    instruction="""
If user wants to remember something,
call remember tool.
""",
    tools=[memory_tool],
)


# ============================================================
# FILE AGENT
# ============================================================

file_agent = LlmAgent(
    name="FileAgent",
    model=MODEL,
    instruction="""
If user wants to save a file,
call save_file tool.
""",
    tools=[file_tool],
)


# ============================================================
# ROUTER AGENT (TRANSFER DEMO)
# ============================================================

router_agent = LlmAgent(
    name="RouterAgent",
    model=MODEL,
    instruction="""
If user mentions:
- remember → transfer to MemoryAgent
- save → transfer to FileAgent
Otherwise answer normally.
""",
    sub_agents=[memory_agent, file_agent],
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    
    runner = Runner(
        agent=router_agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:

            print("\n=================================================")
            print("Event ID      :", event.id)
            print("Author        :", event.author)
            print("Invocation ID :", event.invocation_id)

            if event.content:
                print("Content       :", event.content.parts[0].text)

            if event.actions:
                print("Actions       :", event.actions)

                if event.actions.state_delta:
                    print("🟢 STATE DELTA :", event.actions.state_delta)

                if event.actions.artifact_delta:
                    print("🟣 ARTIFACT DELTA :", event.actions.artifact_delta)

                if event.actions.transfer_to_agent:
                    print("🔄 TRANSFER TO :", event.actions.transfer_to_agent)

                if event.actions.agent_state:
                    print("🧠 AGENT STATE :", event.actions.agent_state)

                if event.actions.end_of_agent:
                    print("🏁 END OF AGENT")

            if event.get_function_calls():
                print("🛠 TOOL CALL EVENT")

            if event.get_function_responses():
                print("🔧 TOOL RESPONSE EVENT")

            if event.is_final_response():
                print("\n✅ FINAL RESPONSE:")
                print(event.content.parts[0].text)

    print("\n=========== ADK COMPLETE EVENT FLOW ===========\n")

    # Normal communication
    await invoke("Hello!")

    # Transfer + state_delta
    await invoke("Remember project as ADK Deep Study")

    # Transfer + artifact_delta
    await invoke("Save report.txt with This is event testing demo")

    # ========================================================
    # SESSION HISTORY
    # ========================================================

    print("\n\n=========== FULL SESSION HISTORY ===========")

    session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    print("Total Events:", len(session.events))

    for i, e in enumerate(session.events):
        print(f"{i+1}. {e.author} | {e.id} | invocation={e.invocation_id}")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()


=========== ADK COMPLETE EVENT FLOW ===========


Event ID      : 2de0753e-dd20-436f-966d-379ecb23819b
Author        : RouterAgent
Invocation ID : e-fb84d21f-2028-4616-b630-ede164c120fe
Content       : Hello! I am a large language model, able to assist with a wide range of tasks and provide information on various topics. How can I help you today?
Actions       : skip_summarization=None state_delta={} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={} requested_tool_confirmations={} compaction=None end_of_agent=None agent_state=None rewind_before_invocation_id=None

✅ FINAL RESPONSE:
Hello! I am a large language model, able to assist with a wide range of tasks and provide information on various topics. How can I help you today?

Event ID      : 819618c4-287e-4c57-9a44-d8b69b347a07
Author        : RouterAgent
Invocation ID : e-21e8bbdc-6b15-4dd1-a4ec-e9424d18d96b
Content       : None
Actions       : skip_summarization=None state_delta={} artifact_delta={} t

## **<font color="blue">Understanding and Using Events</font>**
### ***<font color="green">Identifying Event Origin and Type</font>***
What an event represents by checking:
  - **Who send it?** (`event.author`)
    - `user`, `AgentName`
  - **What's the main payload?** (`event.content` and `event.content.parts`)
    - __Text:__ Indicates a conversation message. `event.content.parts[0].text`
    - __Tool Call Request:__ `event.get_function_calls()`. If not empty, the LLM is asking to execute one or more tools. Each item in the list has `.name` and `.args`.
    - __Tool Result:__ Check `event.get_function_responses()`. If not empty, this event carries the result(s) from tool execution(s). Each item has `.name` and `.response`(the dictionary returned by the tool).
      <font color="brown">_Note:_</font> For history structuring, the `role` inside the `content` is often `user`, but the event `author` is typically the agent that requested the tool call.
  - **Is it streaming output?** (`event.partial`)
    Indicates whether this is an incomplete chunk of text from the LLM.
    - `True`: More text will follow.
    - `False` or `None/Optional.empty()`: This part of the content is complete.

In [14]:
# ============================================================
# ADK EVENT INSPECTOR — COMPLETE UNDERSTANDING DEMO
# Demonstrates:
# - Event origin detection
# - Tool call vs tool result
# - Streaming detection
# - state_delta
# - artifact_delta
# - transfer_to_agent
# - agent stickiness
# - Invocation grouping
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "event_inspector_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOL 1 — STATE TOOL
# ============================================================

async def remember(key: str, value: str, tool_context: ToolContext = None):
    tool_context.state[key] = value
    return {"status": "stored", "key": key, "value": value}


# ============================================================
# TOOL 2 — ARTIFACT TOOL
# ============================================================

async def save_file(filename: str, content: str, tool_context: ToolContext = None):
    text = f"{datetime.utcnow()}\n\n{content}"

    part = types.Part.from_bytes(
        data=text.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return {"status": "saved", "filename": filename, "version": version}


memory_tool = FunctionTool(func=remember)
file_tool = FunctionTool(func=save_file)


# ============================================================
# SUB AGENTS
# ============================================================

memory_agent = LlmAgent(
    name="MemoryAgent",
    model=MODEL,
    instruction="If user wants to remember something, call remember tool.",
    tools=[memory_tool],
)

file_agent = LlmAgent(
    name="FileAgent",
    model=MODEL,
    instruction="If user wants to save file, call save_file tool.",
    tools=[file_tool],
)

router_agent = LlmAgent(
    name="RouterAgent",
    model=MODEL,
    instruction="""
If message contains:
- remember → transfer to MemoryAgent
- save → transfer to FileAgent
Otherwise answer normally.
""",
    sub_agents=[memory_agent, file_agent],
)


# ============================================================
# EVENT CLASSIFIER
# ============================================================

def classify_event(event):

    print("\n--------------------------------------------------")
    print(f"Event ID      : {event.id}")
    print(f"Invocation ID : {event.invocation_id}")
    print(f"Author        : {event.author}")

    # 1️⃣ USER EVENT
    if event.author == "user":
        print("Type: USER MESSAGE")
        if event.content and event.content.parts:
            print("Text:", event.content.parts[0].text)
        return

    # 2️⃣ CONTENT EVENTS
    if event.content and event.content.parts:

        # Tool Call Request
        function_calls = event.get_function_calls()
        if function_calls:
            print("Type: TOOL CALL REQUEST")
            for call in function_calls:
                print("  Tool Name:", call.name)
                print("  Args     :", call.args)
            return

        # Tool Result
        function_responses = event.get_function_responses()
        if function_responses:
            print("Type: TOOL RESULT")
            for response in function_responses:
                print("  Tool Name:", response.name)
                print("  Response :", response.response)
            return

        # Text Message
        text = event.content.parts[0].text
        if text:
            if event.partial:
                print("Type: STREAMING TEXT CHUNK")
            else:
                print("Type: COMPLETE TEXT MESSAGE")
            print("Text:", text)
            return

    # 3️⃣ ACTION EVENTS
    if event.actions:

        if event.actions.transfer_to_agent:
            print("Type: TRANSFER SIGNAL")
            print("Transfer To:", event.actions.transfer_to_agent)

        if event.actions.state_delta:
            print("Type: STATE DELTA")
            print("Delta:", event.actions.state_delta)

        if event.actions.artifact_delta:
            print("Type: ARTIFACT DELTA")
            print("Delta:", event.actions.artifact_delta)

        if event.actions.escalate:
            print("Type: ESCALATION SIGNAL")

        if event.actions.end_of_agent:
            print("Type: END OF AGENT")

        if event.actions.agent_state:
            print("Type: AGENT STATE UPDATE")

        if (
            not event.actions.transfer_to_agent and
            not event.actions.state_delta and
            not event.actions.artifact_delta and
            not event.actions.escalate and
            not event.actions.end_of_agent and
            not event.actions.agent_state
        ):
            print("Type: CONTROL / EMPTY ACTION EVENT")

        return

    print("Type: OTHER / UNKNOWN EVENT")


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=router_agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        print("\n\n================ NEW INVOCATION ================")
        print("USER INPUT:", message)

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            classify_event(event)

            if event.is_final_response():
                print("\n>>> FINAL RESPONSE COMPLETE <<<")

    # -----------------------------
    # TEST FLOW
    # -----------------------------

    await invoke("Hello")
    await asyncio.sleep(1)

    await invoke("Remember project as ADK Mastery")
    await asyncio.sleep(1)

    await invoke("Save report.txt with Event system deep understanding")
    await asyncio.sleep(1)

    # -----------------------------
    # FULL SESSION HISTORY
    # -----------------------------

    print("\n\n=========== FULL SESSION HISTORY ===========")

    session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    print("Total Events:", len(session.events))

    for i, e in enumerate(session.events):
        print(f"{i+1}. {e.author} | {e.id} | invocation={e.invocation_id}")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()



================ NEW INVOCATION ================
USER INPUT: Hello

--------------------------------------------------
Event ID      : b4927fa6-52dc-4145-b074-1ad4fab545d4
Invocation ID : e-9c31ed01-e569-44cf-937d-87e8d1adc705
Author        : RouterAgent
Type: COMPLETE TEXT MESSAGE
Text: Hello! How can I help you today?

>>> FINAL RESPONSE COMPLETE <<<


================ NEW INVOCATION ================
USER INPUT: Remember project as ADK Mastery

--------------------------------------------------
Event ID      : fceb40a3-6cb7-4630-9553-38774aa5fc83
Invocation ID : e-1b4ff499-5538-4c76-ad74-7d05368fec41
Author        : RouterAgent
Type: TOOL CALL REQUEST
  Tool Name: transfer_to_agent
  Args     : {'agent_name': 'MemoryAgent'}

--------------------------------------------------
Event ID      : 0d9f2b7c-7f86-461b-a207-b84ccc760113
Invocation ID : e-1b4ff499-5538-4c76-ad74-7d05368fec41
Author        : RouterAgent
Type: TOOL RESULT
  Tool Name: transfer_to_agent
  Response : {'result': N

### ***<font color="green">Extracting Key Information</font>***
Once we know the event type, access the relevant data:
- __Text Content:__ Always check for the presence of content and parts before accessing text. `text=event.content.parts[0].text`
- __Function Call Details:__
  ```python
    calls = event.get_function_calls()
    if calls:
        for call in calls:
            tool_name = call.name
            arguments = call.args # This is usually a dictionary
            print(f"  Tool: {tool_name}, Args: {arguments}")
            # Application might dispatch execution based on this
  ```
- __Function Response Details:__
  ```python
  responses = event.get_function_responses()
if responses:
    for response in responses:
        tool_name = response.name
        result_dict = response.response # The dictionary returned by the tool
        print(f"  Tool Result: {tool_name} -> {result_dict}")
  ```
- __Identifiers:__
  - `event.id`: Unique ID for this specific event instance.
  - `event.invocation_id`: ID for the entire user-request-to-final-response cycle this event belongs to useful for logging and tracing.


In [15]:
# ============================================================
# ADK EVENT EXTRACTION — COMPLETE TRACE DEMO
# Focus:
# - Extract text safely
# - Extract tool calls (name + args)
# - Extract tool results (response dict)
# - Log event.id
# - Log event.invocation_id
# - Detect state_delta / artifact_delta
# - Detect transfer signals
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "event_extraction_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOLS
# ============================================================

async def remember(key: str, value: str, tool_context: ToolContext = None):
    tool_context.state[key] = value
    return {
        "status": "stored",
        "key": key,
        "value": value
    }


async def save_file(filename: str, content: str, tool_context: ToolContext = None):
    text = f"{datetime.utcnow()}\n\n{content}"

    part = types.Part.from_bytes(
        data=text.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return {
        "status": "saved",
        "filename": filename,
        "version": version
    }


memory_tool = FunctionTool(func=remember)
file_tool = FunctionTool(func=save_file)


# ============================================================
# AGENTS
# ============================================================

memory_agent = LlmAgent(
    name="MemoryAgent",
    model=MODEL,
    instruction="If user wants to remember something, call remember tool.",
    tools=[memory_tool],
)

file_agent = LlmAgent(
    name="FileAgent",
    model=MODEL,
    instruction="If user wants to save a file, call save_file tool.",
    tools=[file_tool],
)

router_agent = LlmAgent(
    name="RouterAgent",
    model=MODEL,
    instruction="""
If message contains:
- remember → transfer to MemoryAgent
- save → transfer to FileAgent
Otherwise respond normally.
""",
    sub_agents=[memory_agent, file_agent],
)


# ============================================================
# EVENT PROCESSOR
# ============================================================

def process_event(event):

    print("\n===================================================")
    print(f"Event ID        : {event.id}")
    print(f"Invocation ID   : {event.invocation_id}")
    print(f"Author          : {event.author}")

    # --------------------------------------------------------
    # TEXT CONTENT EXTRACTION
    # --------------------------------------------------------
    if event.content and event.content.parts:

        # ---- Tool Call Extraction ----
        calls = event.get_function_calls()
        if calls:
            print("Type: TOOL CALL REQUEST")
            for call in calls:
                tool_name = call.name
                arguments = call.args
                print(f"  Tool Name : {tool_name}")
                print(f"  Arguments : {arguments}")
            return

        # ---- Tool Result Extraction ----
        responses = event.get_function_responses()
        if responses:
            print("Type: TOOL RESULT")
            for response in responses:
                tool_name = response.name
                result_dict = response.response
                print(f"  Tool Name : {tool_name}")
                print(f"  Result    : {result_dict}")
            return

        # ---- Text Message Extraction ----
        if len(event.content.parts) > 0:
            text = event.content.parts[0].text
            if text:
                if event.partial:
                    print("Type: STREAMING TEXT CHUNK")
                else:
                    print("Type: COMPLETE TEXT MESSAGE")
                print("Text:", text)
                return

    # --------------------------------------------------------
    # ACTION EXTRACTION
    # --------------------------------------------------------
    if event.actions:

        if event.actions.transfer_to_agent:
            print("Type: TRANSFER SIGNAL")
            print("Transfer To:", event.actions.transfer_to_agent)

        if event.actions.state_delta:
            print("Type: STATE DELTA")
            print("Delta:", event.actions.state_delta)

        if event.actions.artifact_delta:
            print("Type: ARTIFACT DELTA")
            print("Delta:", event.actions.artifact_delta)

        if event.actions.escalate:
            print("Type: ESCALATION SIGNAL")

        if event.actions.agent_state:
            print("Type: AGENT STATE UPDATE")
            print("Agent State:", event.actions.agent_state)

        if event.actions.end_of_agent:
            print("Type: END OF AGENT")

        return

    print("Type: OTHER / CONTROL EVENT")


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=router_agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        print("\n\n================ NEW INVOCATION ================")
        print("USER MESSAGE:", message)

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            process_event(event)

            if event.is_final_response():
                print("\n>>> FINAL RESPONSE COMPLETE <<<")

    # --------------------------------------------------------
    # TEST CASES
    # --------------------------------------------------------

    await invoke("Hello")
    await asyncio.sleep(1)

    await invoke("Remember project as Event Mastery")
    await asyncio.sleep(1)

    await invoke("Save report.txt with Full ADK event tracing")
    await asyncio.sleep(1)

    # --------------------------------------------------------
    # SESSION HISTORY SUMMARY
    # --------------------------------------------------------

    print("\n\n=========== SESSION HISTORY SUMMARY ===========")

    session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    print("Total Events:", len(session.events))

    for i, e in enumerate(session.events):
        print(f"{i+1}. {e.author} | {e.id} | invocation={e.invocation_id}")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()



================ NEW INVOCATION ================
USER MESSAGE: Hello

Event ID        : a21c1cb0-916b-4414-8c47-af75b32f76a2
Invocation ID   : e-4ee5c071-59e6-495a-8530-e5f1cfaa9609
Author          : RouterAgent
Type: COMPLETE TEXT MESSAGE
Text: Hello! How can I help you today?

>>> FINAL RESPONSE COMPLETE <<<


================ NEW INVOCATION ================
USER MESSAGE: Remember project as Event Mastery

Event ID        : f0c58c61-1ecf-43d5-afe1-1bfa0d030378
Invocation ID   : e-3a10bd89-b85f-4ae9-99d3-e7b94ec63123
Author          : RouterAgent
Type: TOOL CALL REQUEST
  Tool Name : transfer_to_agent
  Arguments : {'agent_name': 'MemoryAgent'}

Event ID        : da707e9c-3bfe-47fe-ba18-b91a9c09266b
Invocation ID   : e-3a10bd89-b85f-4ae9-99d3-e7b94ec63123
Author          : RouterAgent
Type: TOOL RESULT
  Tool Name : transfer_to_agent
  Result    : {'result': None}

Event ID        : a9f1b9d2-c219-4ad3-b5f8-f213eff0a8e5
Invocation ID   : e-3a10bd89-b85f-4ae9-99d3-e7b94ec63123
Author 

### ***<font color="green">Detecting Actions and Side Effects</font>***
Then `event.actions` object signals changes that occurred or should occur. Always check if `event.actions` and it's fields/methods exists before accessing them.
- __State Changes:__ Gives you a collection of key-value pairs that were modified in the session state during the step that produced this event.
- `delta = event.actions.state_delta({key:value})`
  ```python
  if event.actions and event.actions.state_delta:
    print(f"  State changes: {event.actions.state_delta}")
    # Update local UI or application state if necessary
  ```
- __Artifact Saves:__ Gives you a collection indicating which artifacts were saved and their new version number (or relevant `Part` information).
  `artifact_changes = event.actions.artifact_delta ({filename: version})`
  ```python
  if event.actions and event.actions.artifact_delta:
    print(f"  Artifacts saved: {event.actions.artifact_delta}")
    # UI might refresh an artifact list
  ```
- __Control Flow Signals:__ Check boolean flag or string values:
  - `event.actions.transfer_to_agent` (string): Control should pass to the named agent.
  - `event.actions.escalate` (bool): A loop should terminate.
  - `event.actions.skip_summarizaiton` (bool): A tool result should not be summarized by the LLM.
    ```python
    if event.actions:
        if event.actions.transfer_to_agent:
            print(f"  Signal: Transfer to {event.actions.transfer_to_agent}")
        if event.actions.escalate:
            print("  Signal: Escalate (terminate loop)")
        if event.actions.skip_summarization:
            print("  Signal: Skip summarization for tool result")
    ```


In [20]:
# ============================================================
# DETECTING ACTIONS AND SIDE EFFECTS (RUNNER VERSION)
# Focused ONLY on event.actions
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "actions_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOLS THAT CREATE SIDE EFFECTS
# ============================================================

# ---- STATE CHANGE TOOL ----
async def remember(key: str, value: str, tool_context: ToolContext = None):
    tool_context.state[key] = value
    return {"stored": {key: value}}

# ---- ARTIFACT SAVE TOOL ----
async def save_file(filename: str, content: str, tool_context: ToolContext = None):
    part = types.Part.from_bytes(
        data=content.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return {"saved": filename, "version": version}


memory_tool = FunctionTool(func=remember)
file_tool = FunctionTool(func=save_file)


# ============================================================
# AGENTS
# ============================================================

memory_agent = LlmAgent(
    name="MemoryAgent",
    model=MODEL,
    instruction="If user says remember, call remember tool.",
    tools=[memory_tool],
)

file_agent = LlmAgent(
    name="FileAgent",
    model=MODEL,
    instruction="If user says save, call save_file tool.",
    tools=[file_tool],
)

router_agent = LlmAgent(
    name="RouterAgent",
    model=MODEL,
    instruction="""
If message contains:
- remember → transfer to MemoryAgent
- save → transfer to FileAgent
- exit → escalate
Otherwise respond normally.
""",
    sub_agents=[memory_agent, file_agent],
)


# ============================================================
# ACTION DETECTION LOGIC
# ============================================================

def detect_actions(event):

    # ALWAYS check event.actions exists first
    if not event.actions:
        return

    print("\n--- ACTIONS DETECTED ---")

    # ---------------------------
    # STATE DELTA
    # ---------------------------
    if event.actions.state_delta:
        print("State changes:", event.actions.state_delta)

    # ---------------------------
    # ARTIFACT DELTA
    # ---------------------------
    if event.actions.artifact_delta:
        print("Artifacts saved:", event.actions.artifact_delta)

    # ---------------------------
    # TRANSFER SIGNAL
    # ---------------------------
    if event.actions.transfer_to_agent:
        print("Signal: Transfer to", event.actions.transfer_to_agent)

    # ---------------------------
    # ESCALATION
    # ---------------------------
    if event.actions.escalate:
        print("Signal: Escalate (terminate loop)")

    # ---------------------------
    # SKIP SUMMARIZATION
    # ---------------------------
    if event.actions.skip_summarization:
        print("Signal: Skip summarization for tool result")


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=router_agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        print("\n==============================")
        print("USER:", message)

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:

            # Print normal text
            if event.content and event.content.parts:
                text = event.content.parts[0].text
                if text:
                    print("Agent:", text)

            # Detect side effects
            detect_actions(event)

            if event.is_final_response():
                print(">>> FINAL RESPONSE <<<")

    # TEST CASES
    await invoke("Hello")
    await asyncio.sleep(1)

    await invoke("remember project as ADK")
    await asyncio.sleep(1)

    await invoke("save demo.txt with Side effects example")
    await asyncio.sleep(1)

    await invoke("exit")


# ============================================================
# EXECUTION (Jupyter)
# ============================================================

await run_demo()


USER: Hello
Agent: Hello! How can I help you today?

--- ACTIONS DETECTED ---
>>> FINAL RESPONSE <<<

USER: remember project as ADK

--- ACTIONS DETECTED ---

--- ACTIONS DETECTED ---
Signal: Transfer to MemoryAgent

--- ACTIONS DETECTED ---

--- ACTIONS DETECTED ---
State changes: {'project': 'ADK'}
Agent: OK. I'll remember that project is ADK.


--- ACTIONS DETECTED ---
>>> FINAL RESPONSE <<<

USER: save demo.txt with Side effects example

--- ACTIONS DETECTED ---

--- ACTIONS DETECTED ---
Signal: Transfer to FileAgent

--- ACTIONS DETECTED ---

--- ACTIONS DETECTED ---
Artifacts saved: {'demo.txt': 0}
Agent: OK. I saved that to `demo.txt`.

--- ACTIONS DETECTED ---
>>> FINAL RESPONSE <<<

USER: exit

--- ACTIONS DETECTED ---

--- ACTIONS DETECTED ---
Signal: Transfer to RouterAgent
Agent: I've received an "exit" command. I'm escalating this request.

--- ACTIONS DETECTED ---
>>> FINAL RESPONSE <<<


In [21]:
# ============================================================
# DETECTING ACTIONS & SIDE EFFECTS — RUNNER VERSION
# Compatible with Your Working ADK Setup
# ============================================================

import os
import asyncio
from datetime import datetime

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "actions_side_effects_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOLS
# ============================================================

# ---- STATE MODIFICATION TOOL ----
async def remember(key: str, value: str, tool_context: ToolContext = None):
    tool_context.state[key] = value
    return {"status": "stored", "key": key, "value": value}


# ---- ARTIFACT SAVE TOOL ----
async def save_file(filename: str, content: str, tool_context: ToolContext = None):
    text = f"{datetime.utcnow()}\n\n{content}"

    part = types.Part.from_bytes(
        data=text.encode(),
        mime_type="text/plain"
    )

    version = await tool_context.save_artifact(
        filename=filename,
        artifact=part
    )

    return {"status": "saved", "filename": filename, "version": version}


memory_tool = FunctionTool(func=remember)
file_tool = FunctionTool(func=save_file)


# ============================================================
# AGENTS
# ============================================================

memory_agent = LlmAgent(
    name="MemoryAgent",
    model=MODEL,
    instruction="If user says remember, call remember tool.",
    tools=[memory_tool],
)

file_agent = LlmAgent(
    name="FileAgent",
    model=MODEL,
    instruction="If user says save, call save_file tool.",
    tools=[file_tool],
)

router_agent = LlmAgent(
    name="RouterAgent",
    model=MODEL,
    instruction="""
If message contains:
- remember → transfer to MemoryAgent
- save → transfer to FileAgent
- exit → escalate
Otherwise respond normally.
""",
    sub_agents=[memory_agent, file_agent],
)


# ============================================================
# EVENT PROCESSOR
# ============================================================

def process_event(event):

    print("\n--------------------------------------------------")
    print(f"Event ID      : {event.id}")
    print(f"Invocation ID : {event.invocation_id}")
    print(f"Author        : {event.author}")

    # --------------------------------------------------------
    # CONTENT EXTRACTION
    # --------------------------------------------------------

    if event.content and event.content.parts:

        calls = event.get_function_calls()
        if calls:
            print("Type: TOOL CALL")
            for call in calls:
                print("  Tool:", call.name)
                print("  Args:", call.args)
            return

        responses = event.get_function_responses()
        if responses:
            print("Type: TOOL RESULT")
            for r in responses:
                print("  Tool:", r.name)
                print("  Result:", r.response)
            return

        text = event.content.parts[0].text
        if text:
            print("Type: TEXT")
            print("Text:", text)
            return

    # --------------------------------------------------------
    # ACTION DETECTION (Main Focus)
    # --------------------------------------------------------

    if event.actions:

        print("Type: ACTION EVENT")

        if event.actions.transfer_to_agent:
            print("  🔁 Transfer To:", event.actions.transfer_to_agent)

        if event.actions.state_delta:
            print("  🧠 State Delta:", event.actions.state_delta)

        if event.actions.artifact_delta:
            print("  📂 Artifact Delta:", event.actions.artifact_delta)

        if event.actions.escalate:
            print("  ⛔ Escalation Signal")

        if event.actions.agent_state:
            print("  ⚙ Agent State:", event.actions.agent_state)

        if event.actions.end_of_agent:
            print("  🏁 End Of Agent")

        return

    print("Type: OTHER EVENT")


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=router_agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        print("\n\n================ NEW INVOCATION ================")
        print("USER:", message)

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:
            process_event(event)

            if event.is_final_response():
                print("\n>>> FINAL RESPONSE COMPLETE <<<")

    # ---- TESTS ----

    await invoke("Hello")
    await asyncio.sleep(1)

    await invoke("remember project as ADK Mastery")
    await asyncio.sleep(1)

    await invoke("save demo.txt with Full side effect tracing")
    await asyncio.sleep(1)

    await invoke("exit")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()



================ NEW INVOCATION ================
USER: Hello

--------------------------------------------------
Event ID      : af1a4dd0-d320-44a2-b0fe-5b07e8953f71
Invocation ID : e-b63b9d2c-ec79-44fb-8442-c55e5040fe2f
Author        : RouterAgent
Type: TEXT
Text: Hello! How can I help you today?

>>> FINAL RESPONSE COMPLETE <<<


================ NEW INVOCATION ================
USER: remember project as ADK Mastery

--------------------------------------------------
Event ID      : 4ab0192b-de7b-4699-b96d-31c18f083e32
Invocation ID : e-726170e5-e07a-41e6-a081-2aba9ed9dbfd
Author        : RouterAgent
Type: ACTION EVENT

>>> FINAL RESPONSE COMPLETE <<<


================ NEW INVOCATION ================
USER: save demo.txt with Full side effect tracing

--------------------------------------------------
Event ID      : e2c5666f-854e-4ebd-a811-6accfbf21aa1
Invocation ID : e-aba04910-e5bd-4614-abb2-73a9eb40874e
Author        : RouterAgent
Type: TOOL CALL
  Tool: transfer_to_agent
  Args

### ***<font color="green">Determining if an Event is a "Final" Response</font>***
Use the built-in helper method `event.is_final_response()` to identify events suitable for display as the agent's complete output for a turn.
- __Purpose:__ Filters out intermediate steps (like tool calls, partial streaming text, internal state updates) from the final user-facing message(s).
- __When `True`:__
  1. The event contains a tool result (`function_response`) and `skip_summarization` is `True`.
  2. The event contains a tool call (`function_call`) for a tool marked as `is_long_running=True`.
  3. OR, **all** of the following are met:
     - No function calls (`get_function_calls()` is empty).
     - No function response (`get_function_response()` is empty).
     - No a partial stream chunk (`partial` is not `True`).
     - Doesn't end with a code execution result that might need further processing/display.
- __Usage:__ Filter the event stream in your application logic.


In [23]:
# ============================================================
# FINAL RESPONSE DETECTION (Compatible with Your ADK)
# No skip_summarization / is_long_running flags
# ============================================================

import os
import asyncio

from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
import google.genai.types as types

from config import config


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "final_response_demo"
USER_ID = "user_1"
SESSION_ID = "session_1"
MODEL = "gemini-2.5-flash"


# ============================================================
# TOOL
# ============================================================

async def structured_tool(tool_context: ToolContext = None):
    return {"status": "success", "message": "Structured tool output"}

tool = FunctionTool(func=structured_tool)


# ============================================================
# AGENT
# ============================================================

agent = LlmAgent(
    name="FinalResponseAgent",
    model=MODEL,
    instruction="""
If user says 'tool', call structured_tool.
Otherwise answer normally.
""",
    tools=[tool],
)


# ============================================================
# RUN DEMO
# ============================================================

async def run_demo():

    session_service = InMemorySessionService()
    artifact_service = InMemoryArtifactService()

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service,
        artifact_service=artifact_service
    )

    async def invoke(message):

        print("\n===================================")
        print("USER:", message)

        content = types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )

        full_response_text = ""

        events = runner.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        async for event in events:

            # ----------------------------
            # Accumulate streaming text
            # ----------------------------
            if (
                event.partial
                and event.content
                and event.content.parts
                and event.content.parts[0].text
            ):
                full_response_text += event.content.parts[0].text

            # ----------------------------
            # Final Response Detection
            # ----------------------------
            if event.is_final_response():

                print("\n--- FINAL OUTPUT DETECTED ---")

                # CASE 1: Normal text response
                if (
                    event.content
                    and event.content.parts
                    and event.content.parts[0].text
                ):
                    final_text = full_response_text + (
                        event.content.parts[0].text
                        if not event.partial else ""
                    )
                    print("Display to user:")
                    print(final_text.strip())
                    full_response_text = ""

                # CASE 2: Tool response
                elif event.get_function_responses():
                    response_data = event.get_function_responses()[0].response
                    print("Display tool result:")
                    print(response_data)

                else:
                    print("Final non-textual response")

    # TEST CASES
    await invoke("Hello")
    await asyncio.sleep(1)

    await invoke("tool")


# ============================================================
# EXECUTION
# ============================================================

await run_demo()


USER: Hello

--- FINAL OUTPUT DETECTED ---
Display to user:
Hello! How can I help you today?

USER: tool

--- FINAL OUTPUT DETECTED ---
Display to user:
I have executed the structured tool. It returned a message: "Structured tool output" with a status of "success".


## **<font color="blue">How Events Flow: Generating and Processing</font>**
## **<font color="blue">Common Event Examples></font>**
## **<font color="blue">Additional Context and Event Details</font>**